※ Python3을 이용해주세요.
Python3을 이용하려면 , 메뉴바에서［런타임］-［런타임 유형 변경을 선택하면 표시되는［런타임 유형］란에서 「Python 3」에 선택하 저장을 눌러 주세요.

In [ ]:
# I. 신경망 구조 이해를 위한 사전 준비

## ■ （1） 데이터 준비

In [0]:
# Google Colab에서 최신 2.x를 사용할 경우 2.x로 전환
% tensorflow_version 2.x

In [0]:
import sys
print('Python', sys.version)
# Python 3.6.8 (default, Jan 14 2019, 11:02:34) ...

### 리스트 1 좌표점 데이터를 생성하는 라이브러리 설치

In [0]:
!pip install playground-data

## ■ （2）문제 유형

### 리스트 2 문제 유형과 데이터 유형 선택

In [0]:
# playground-data 라이브러리의 plygdata 패키지를 pg라는 별칭으로 불러오기
import plygdata as pg

# 문제유형은 분류(Classification)를 선택하고,
# 데이터 유형은 2개의 가우시안 데이터(TwoGauss Data)를 선택하는 경우,
# 설정값을 정수로 정의
PROBLEM_DATA_TYPE = pg.DatasetType.ClassifyTwoGaussData

# 실제 데이터 생성은 후술하는 pg.generate_data()함수의 호출에서 실시한다

## ■ （3）전처리

### 리스트 3 전처리에서의 데이터 분할

In [0]:
# 각종 설정을 상수로 정의
TRAINING_DATA_RATIO = 0.5  # 데이터의 몇 %를 훈련용, 나머지는 정밀도 검증용 ： 50％
DATA_NOISE = 0.0           # 노이즈： 0％

# 이미 정의된 상수를 인수로 지정하여 데이터를 생성한다
data_list = pg.generate_data(PROBLEM_DATA_TYPE, DATA_NOISE)

# 데이터를 훈련용과 정밀도 검증용을 지정 비율로 분할하고, 각각 데이터(X)와 교사 라벨(y)로 나눈다
X_train, y_train, X_valid, y_valid = pg.split_data(data_list, training_size=TRAINING_DATA_RATIO)

### 【체크】 데이터 분할 후의 각 변수의 내용 예

In [0]:
# 각각 5건씩 출력
print('X_train:'); print(X_train[:5])
print('y_train:'); print(y_train[:5])
print('X_valid:'); print(X_valid[:5])
print('y_valid:'); print(y_valid[:5])

# II. 신경망 구조 이해

## ■ （4） 방법의 선택 및 모델 정의: 뉴런

### 활성화 함수 그래프

In [0]:
#@title Sigmoid 함수
# This code will be hidden when the notebook is loaded.

import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1.0 / (1.0 + np.exp(-x))

x = np.arange(-6.0, 6.0, 0.001)
plt.plot(x, sigmoid(x), label = "Sigmoid")
plt.xlim(-6, 6)
plt.ylim(-0.2, 1.2)
plt.grid()
plt.legend()
plt.show()

In [0]:
#@title tanh 함수
# This code will be hidden when the notebook is loaded.

import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1.0 / (1.0 + np.exp(-x))

def tanh(x):
  return np.tanh(x)

x = np.arange(-6.0, 6.0, 0.001)
plt.plot(x, sigmoid(x), label = "Sigmoid")
plt.plot(x, tanh(x), label = "tanh")
plt.xlim(-6, 6)
plt.ylim(-1.2, 1.2)
plt.grid()
plt.legend()
plt.show()

### 【체크】 TensorFlow 버전
기본적으로 Colab에 설치된 것을 사용한다. 만약 2점대로 되어 있는 경우는, 리스트 4-0을 실행해 버전 2.0을 사용한다.

In [0]:
import tensorflow as tf
print('TensorFlow', tf.__version__)

### 리스트 4-0 [옵션] 라이브러리 TensorFlow최신 버전 설치

In [0]:
# Google Colab에서 최신 2.x를 사용하는 경우
% tensorflow_version 2.x

# 최신 버전으로 업그레이드하는 경우
!pip install --upgrade tensorflow

# 버전을 명시하고 업그레이드하는 경우
#!pip install --upgrade tensorflow===2.0.0

# 최신 버전을 설치하는 경우
#!pip install tensorflow

# 버전을 명시하고 설치하는 경우
#!pip install tensorflow===2.0.0

### [옵션] [체크] TensorFlow 버전(※ 설치 후 확인)
버전 2.x가 있는지 다시 확인합니다.

In [0]:
import tensorflow as tf
print('TensorFlow', tf.__version__)
# TensorFlow 2.0.0 ...

### 리스트 4-1　뉴런의 모델 설계

In [0]:
# 라이브러리 TensorFlow의 tensorflow 패키지를 tf라는 별칭으로 가져오기
import tensorflow as tf
# 라이브러리 NumPy의 numpy 패키지를 "np"라는 별칭으로 가져오기
import numpy as np

# 상수(모델 정의 시에 필요한 것)
INPUT_FEATURES = 2  # 입력(특징) 수： 2
LAYER1_NEURONS = 1  # 뉴런 수： 1

# 매개변수(뉴런에 대한 입력에 필요한 것)
weight_array = np.array([[ 0.6 ],
                         [-0.2 ]])       # 가중치
bias_array   = np.array([  0.8 ])        # 바이어스

# 적층형 모델 정의
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(
    input_shape=(INPUT_FEATURES,),       # 입력 특징
    units=LAYER1_NEURONS,                # 유닛 수
    weights=[weight_array, bias_array],  # 가중치와 바이어스의 초기값
    activation='tanh')                   # 활성화 함수
])

# 이 모델에 데이터를 입력하여 출력을 얻는다(예측: predict)
X_data = np.array([[1.0, 2.0]])          # 입력할 좌표 데이터(1.0, 2.0）
print(model.predict(X_data))             # 출력 얻기
# [[0.7615942]] ……

## ■ （4） 방법의 선택과 모델 정의 : 신경망

### 리스트 4-2　신경망의 모델 설계

In [0]:
# 라이브러리 TensorFlow의 tensorflow 패키지를 tf라는 별칭으로 가져오기
import tensorflow as tf
# 라이브러리 NumPy의 numpy 패키지를 np라는 별칭으로 가져오기
import numpy as np

# 상수(모델 정의 시 필요한 숫자)
INPUT_FEATURES = 2  # 입력(특징) 수 ： 2
LAYER1_NEURONS = 3  # 뉴런 수 : 3
LAYER2_NEURONS = 3  # 뉴런 수 ： 3
OUTPUT_RESULTS = 1  # 출력 결과 수 ： 1

# 가중치와 바이어스 매개변수를 지정하지 않음(일반적으로 지정하지 않음)

# 적층형 모델 정의
model = tf.keras.models.Sequential([
  # 숨겨진 레이어: 첫 번째 레이어
  tf.keras.layers.Dense(
    input_shape=(INPUT_FEATURES,),       # 입력(입력 레이어)
    units=LAYER1_NEURONS,                # 유닛 수
    activation='tanh'),                  # 활성화 함수
  # 숨겨진 레이어: 두 번째 레이어
  tf.keras.layers.Dense(
    units=LAYER2_NEURONS,                # 유닛 수
    activation='tanh'),                  # 활성화 함수
  # 출력 레이어
  tf.keras.layers.Dense(
    units=OUTPUT_RESULTS,                 # 유닛 수
    activation='tanh'),                  # 활성화 함수
])

In [0]:
model.summary()

## ■ （4） 기법 선택 및 모델 정의: 활성화 함수

### 활성화 함수 그래프

In [0]:
#@title 선형 함수
# This code will be hidden when the notebook is loaded.

import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1.0 / (1.0 + np.exp(-x))

def linear(x):
  return x

x = np.arange(-6.0, 6.0, 0.001)
plt.plot(x, sigmoid(x), label = "Sigmoid")
plt.plot(x, linear(x), label = "Linear")
plt.xlim(-6, 6)
plt.ylim(-1.2, 1.2)
plt.grid()
plt.legend()
plt.show()

In [0]:
#@title ReLU 함수
# This code will be hidden when the notebook is loaded.

import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1.0 / (1.0 + np.exp(-x))

def relu(x):
  return np.maximum(0, x)

x = np.arange(-6.0, 6.0, 0.001)
plt.plot(x, sigmoid(x), label = "Sigmoid")
plt.plot(x, relu(x), label = "ReLU")
plt.xlim(-6, 6)
plt.ylim(-1.2, 1.2)
plt.grid()
plt.legend()
plt.show()

### 리스트 4-3 활성화 함수 지정 예

In [0]:
# 라이브러리 TensorFlow의 tensorflow 패키지를 tf라는 별칭으로 가져오기
import tensorflow as tf
# 라이브러리 NumPy의 numpy 패키지를 np라는 별칭으로 가져오기
import numpy as np

# 상수(모델 정의 시 필요한 숫자)
INPUT_FEATURES = 2      # 입력(특징) 수 ： 2
LAYER1_NEURONS = 3      # 뉴런 수 ： 3
LAYER2_NEURONS = 3      # 뉴런 수 ： 3
OUTPUT_RESULTS = 1      # 출력 결과 수： 1
ACTIVATION = 'sigmoid'  # 활성화 함수 : sigmoid 함수

# 적층형 모델 정의
model = tf.keras.models.Sequential([
  # 숨겨진 레이어: 첫 번째 레이어
  tf.keras.layers.Dense(
    input_shape=(INPUT_FEATURES,),       # 입력(입력 레이어)
    units=LAYER1_NEURONS,                # 유닛 수
    activation=ACTIVATION),              # 활성화 함수
  # 숨겨진 레이어: 두 번째 레이어
  tf.keras.layers.Dense(
    units=LAYER2_NEURONS,                # 유닛 수
    activation=ACTIVATION),              # 활성화 함수
  # 출력 레이어
  tf.keras.layers.Dense(
    units=OUTPUT_RESULTS,                # 유닛 수
    activation='tanh'),                  # 활성화 함수
])

## ■ （4） 기법의 선택과 모델 정의 : 정규화

### 리스트 4-4　정규화의 지정 예

In [0]:
# 라이브러리 TensorFlow의 tensorflow 패키지를 tf라는 별칭으로 가져오기
import tensorflow as tf
# 라이브러리 NumPy의 numpy 패키지를 np라는 별칭으로 가져오기
import numpy as np

# 상수(모델 정의 시에 필요한 것)
INPUT_FEATURES = 2      # 입력(특징) 수 ： 2
LAYER1_NEURONS = 3      # 뉴런 수： 3
LAYER2_NEURONS = 3      # 뉴런 수： 3
OUTPUT_RESULTS = 1      # 출력 결과 수 ： 1
ACTIVATION = 'tanh'     # 활성화 함수 : tanh 함수
REGULARIZATION = tf.keras.regularizers.l2(0.03)  # 정규화: L2, 정규화율: 0.03

# 적층형 모델 정의
model = tf.keras.models.Sequential([
  # 숨겨진 레이어: 첫 번째 레이어
  tf.keras.layers.Dense(
    input_shape=(INPUT_FEATURES,),        # 입력(입력 레이어)
    units=LAYER1_NEURONS,                 # 유닛 수
    activation=ACTIVATION,                # 활성화 함수
    kernel_regularizer= REGULARIZATION),  # 정규화
                  
  # 숨겨진 레이어: 두 번째 레이어
  tf.keras.layers.Dense(
    units=LAYER2_NEURONS,                 # 유닛 수
    activation=ACTIVATION,                # 활성화 함수
    kernel_regularizer= REGULARIZATION),  # 정규화

  # 출력 레이어
  tf.keras.layers.Dense(
    units=OUTPUT_RESULTS,                 # 유닛 수
    activation='tanh',                    # 활성화 함수
    kernel_regularizer= REGULARIZATION)   # 정규화    
])

## ■ （5） 학습 방법 설계 및 모델 생성: 손실 함수

### 리스트 5-1　손실 함수 정의

In [0]:
# 상수(학습 방법 설계시에 필요한 수치)
LOSS = 'mean_squared_error'  # 손실 함수: 평균 제곱 오차

## ■ （5） 학습 방법 설계 및 모델 생성: 최적화

### 리스트 5-2 최적화 정의

In [0]:
# (필요한 경우) TensorFlow v2의 최신 버전으로 업그레이드해야 함
#!pip install --upgrade tensorflow
# 라이브러리 TensorFlow의 tensorflow 패키지를 tf라는 별칭으로 가져오기
import tensorflow as tf

# 상수(학습 방법 설계시에 필요한 것)
OPTIMIZER = tf.keras.optimizers.SGD  # 최적화: 확률적 경사 하강법

## ■ （5） 학습 방법 설계 및 모델 생성: 학습률

### 리스트 5-3　학습률 정의

In [0]:
# 상수(학습 방법 설계시에 필요한 것)
LEARNING_RATE = 0.03     # 학습률 : 0.03

### 리스트 5-4 [전회 복습] 모델 정의

In [0]:
# 라이브러리 "TensorFlow"의 tensorflow 패키지를 "tf"라는 별칭으로 가져오기
import tensorflow as tf

# 상수(모델 정의 시 필요한 숫자)
INPUT_FEATURES = 2      # 입력(특징) 수 ： 2
LAYER1_NEURONS = 3      # 뉴런 수 ： 3
LAYER2_NEURONS = 3      # 뉴런 수 ： 3
OUTPUT_RESULTS = 1      # 출력 결과 수 ： 1
ACTIVATION = 'tanh'     # 활성화 함수： tanh 함수

# 적층형 모델 정의
model = tf.keras.models.Sequential([

  # 숨겨진 레이어: 첫 번째 레이어
  tf.keras.layers.Dense(
    input_shape=(INPUT_FEATURES,),       # 입력 레이어
    units=LAYER1_NEURONS,                # 유닛 수
    activation=ACTIVATION),              # 활성화 함수

  # 숨겨진 레이어: 두 번째 레이어
  tf.keras.layers.Dense(
    units=LAYER2_NEURONS,                # 유닛 수
    activation=ACTIVATION),              # 활성화 함수

  # 출력 레이어
  tf.keras.layers.Dense(
    units=OUTPUT_RESULTS,                # 유닛 수
    activation='tanh'),                  # 활성화 함수
])

In [0]:
model.summary()

### 리스트 5-5　정답률 (정확도)의 맞춤 지표

In [0]:
import tensorflow.keras.backend as K

def tanh_accuracy(y_true, y_pred):           # y_true는 정답, y_pred은 예측(출력)
  threshold = K.cast(0.0, y_pred.dtype)      # -1 또는 1을 구분하는 임계값을 작성
  y_pred = K.cast(y_pred >= threshold, y_pred.dtype) # 임계값 미만에서 0, 이상에서 1로 변환
  # 2배로 -1.0함으로써 0/1을 -1.0/1.0으로 스케일 변환하여 정답률을 계산
  return K.mean(K.equal(y_true, y_pred * 2 - 1.0), axis=-1)

### 리스트 5-6　모델 생성

In [0]:
# 상수(학습 방법 설계시에 필요한 것)
LOSS = 'mean_squared_error'          # 손실 함수: 평균 제곱 오차
OPTIMIZER = tf.keras.optimizers.SGD  # 최적화: 확률적 경사 하강법
LEARNING_RATE = 0.03                 # 학습률 ： 0.03

# 모델 생성
model.compile(optimizer=OPTIMIZER(learning_rate=LEARNING_RATE),
              loss=LOSS,
              metrics=[tanh_accuracy])  # 정밀도(정답률)

## ■ （6） 학습: 훈련

### 리스트 6-1　[이전 회복] 데이터 취득 및 분할

In [0]:
# (필요한 경우) 좌표 점 데이터를 생성하는 라이브러리를 설치해야 함
#!pip install playground-data

# playground-data 라이브러리의 plygdata 패키지를 "pg"라는 별칭으로 가져오기
import plygdata as pg

# 문제 유형에서 분류를 선택하고,
# 데이터 유형으로 2개의 가우시안 데이터(TwoGaussData)를 선택하는 경우,
# 설정값을 상수로 정의
PROBLEM_DATA_TYPE = pg.DatasetType.ClassifyTwoGaussData

# 다양한 설정을 상수로 정의
TRAINING_DATA_RATIO = 0.5  # 데이터의 몇 %를 훈련용으로? (남은 정밀도 검증용) ： 50%
DATA_NOISE = 0.0           # 노이즈 : 0％

# 미리 정의된 상수를 인수로 지정하여 데이터 생성
data_list = pg.generate_data(PROBLEM_DATA_TYPE, DATA_NOISE)

# 데이터를 훈련용과 정밀도 검증용을 지정 비율로 나누고, 각각을 데이터(X)와 교사 라벨(y)로 나눈다.
X_train, y_train, X_valid, y_valid = pg.split_data(data_list, training_size=TRAINING_DATA_RATIO)

In [0]:
# 각각 5건씩 출력
print('X_train:'); print(X_train[:5])
print('y_train:'); print(y_train[:5])
print('X_valid:'); print(X_valid[:5])
print('y_valid:'); print(y_valid[:5])

### 리스트 6-2　학습

In [0]:
# 상수(학습 방법 설계시에 필요한 것)
BATCH_SIZE = 1   # 배치 사이즈: 1(선택사항은 1 ~ 30)
EPOCHS = 100     # 에포크 수 ： 100

# 학습(다음의 리스트 6-3 미니 배치 학습으로 실행하기 때문에, 여기에서는 아직 실행하지 않음)
#hist = model.fit(x=X_train,                          # 교육용 데이터
#                 y=y_train,                          # 훈련용 라벨
#                 validation_data=(X_valid, y_valid), # 정밀도 검증용
#                 batch_size=BATCH_SIZE,              # 배치 크기
#                 epochs=EPOCHS,                      # 에포크 수
#                 verbose=1)                          # 실행 상태 표시

## ■ （6）학습: 배치 크기

### 리스트 6-3　미니 배치 학습

In [0]:
# 상수(학습 방법 설계시에 필요한 것)
BATCH_SIZE = 15  # 배치 사이즈: 15(선택사항은 1 ~ 30)
EPOCHS = 100     # 에포크 수： 100

# 학습
hist = model.fit(x=X_train,                          # 교육용 데이터
                 y=y_train,                          # 훈련용 라벨
                 validation_data=(X_valid, y_valid), # 정밀도 검증용
                 batch_size=BATCH_SIZE,              # 배치 크기
                 epochs=EPOCHS,                      # 에포크 수
                 verbose=1)                          # 실행 상태 표시

## ■ （7） 평가: 손실 그래프

### 리스트 7-1　손실 값의 추이 ​​그래프 그리기

In [0]:
import matplotlib.pyplot as plt

# 학습 결과(손실) 그래프를 그리기
train_loss = hist.history['loss']
valid_loss = hist.history['val_loss']
epochs = len(train_loss)
plt.plot(range(epochs), train_loss, marker='.', label='loss (Training data)')
plt.plot(range(epochs), valid_loss, marker='.', label='loss (Validation data)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

### 리스트 7-2　콜백 지정 (조기 종료 및 CSV 로그 출력)

In [0]:
# 조기 종료
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

# CSV 로거
csv_logger = tf.keras.callbacks.CSVLogger('training.log')

# 학습
hist = model.fit(x=X_train,                          # 교육용 데이터
                 y=y_train,                          # 훈련용 라벨
                 validation_data=(X_valid, y_valid), # 정밀도 검증용
                 batch_size=BATCH_SIZE,              # 배치 크기
                 epochs=EPOCHS,                      # 에포크 수
                 verbose=1,                          # 실행 상태 표시
                 callbacks=[es, csv_logger])         # 콜백

In [0]:
# CSV 로그 출력 파일 다운로드
from google.colab import files
files.download('training.log')

## ■ （8） 테스트 : 알 수 없는 데이터로 추론 및 평가

### 리스트 8-1　미지의 데이터에 의한 테스트(추론과 평가)

In [0]:
import plygdata as pg
import numpy as np

# 알 수 없는 테스트 데이터 생성
PROBLEM_DATA_TYPE = pg.DatasetType.ClassifyTwoGaussData
TEST_DATA_RATIO = 1.0  # 데이터의 몇 %를 훈련용으로? (남은 정밀도 검증용) ： 100%
DATA_NOISE = 0.0       # 노이즈： 0％
data_list = pg.generate_data(PROBLEM_DATA_TYPE, DATA_NOISE)
X_test, y_test, _, _ = pg.split_data(data_list, training_size=TEST_DATA_RATIO)

# 학습된 모델을 이용한 추론
result_proba = model.predict(X_test)
result_class = np.frompyfunc(lambda x: 1 if x >= 0.0 else -1, 1, 1)(result_proba) # 이산화
# 각각 5건씩 출력
print('proba:'); print(result_proba[:5])  # 예측
print('class:'); print(result_class[:5])  # 분류

# 알 수 없는 테스트 데이터로 학습된 모델의 일반화 성능 평가
score = model.evaluate(X_test, y_test)
print('test loss:', score[0])  # 손실
print('test acc:', score[1])   # 정답률

## ■ (9) 기타

### 리스트 9-1　신경망의 가중치와 바이어스를 조사하기위한 샘플 코드

In [0]:
# 가중치와 바이어스를 함께 수집
model.get_weights()

In [0]:
start_index = 0  # Sequential(적층형) 모델의 경우 0 시작
#start_index = 1 # Functional(함수 유형) API의 경우 0이 입력 레이어이므로 주의


# 숨겨진 레이어 1
hidden1_layer = model.layers[start_index]
print('숨겨진 레이어 1 ：',hidden1_layer.name)
hidden1_weights =  hidden1_layer.get_weights()
print(hidden1_weights)
# weights = hidden1_weights[0]
h1_w1 = hidden1_weights[0][0, 0]
h1_w2 = hidden1_weights[0][1, 0]
# biases = hidden1_weights[1]
h1_b = hidden1_weights[1][0]
# 일부 출력
print('h1_w1=',h1_w1)
print('h1_w2=',h1_w2)
print('h1_b=',h1_b)
print()

# 중간 레이어의 출력 예
print('숨겨진 레이어 1 출력')
test_data = np.array([[0.4, 3.2], [5.8, 3.1], [-5.8, -3.1]], dtype=np.float32)
intermediate1_model = tf.keras.models.Model(inputs=model.input, outputs=hidden1_layer.output)
intermediate1_output = intermediate1_model.predict(test_data)
print(intermediate1_output)
print()


# 숨겨진 레이어 2
hidden2_layer = model.layers[start_index+1]
print('숨겨진 레이어 2 ：',hidden2_layer.name)
hidden2_weights =  hidden2_layer.get_weights()
print(hidden2_weights)
# weights = hidden2_weights[0]
h2_w1 = hidden2_weights[0][0, 0]
h2_w2 = hidden2_weights[0][1, 0]
# biases = hidden2_weights[1]
h2_b = hidden2_weights[1][0]
# 일부 출력
print('h2_w1=',h2_w1)
print('h2_w2=',h2_w2)
print('h2_b=',h2_b)
print()

# 중간 레이어의 출력 예
print('숨겨진 레이어 2 출력')
test_data = np.array([[0.4, 3.2], [5.8, 3.1], [-5.8, -3.1]], dtype=np.float32)
intermediate2_model = tf.keras.models.Model(inputs=model.input, outputs=hidden2_layer.output)
intermediate2_output = intermediate2_model.predict(test_data)
print(intermediate2_output)
print()


# 출력 레이어
output_layer = model.layers[start_index+2]
print('출력 레이어 : ',output_layer.name)
output_weights =  output_layer.get_weights()
print(output_weights)
# weights = output_weights[0]
o_w1 = output_weights[0][0, 0]
o_w2 = output_weights[0][1, 0]
# biases = output_weights[1]
o_b = output_weights[1][0]
# 일부 출력
print('o_w1=',o_w1)
print('o_w2=',o_w2)
print('o_b=',o_b)
print()

# 출력 레이어의 출력 예
print('출력 레이어 출력 = 모델 전체 출력')
output_model = tf.keras.models.Model(inputs=model.input, outputs=output_layer.output)
output_output = output_model.predict(test_data)
print(output_output)
print()

print('모델 전체 출력')
test_data = np.array([[0.4, 3.2], [5.8, 3.1], [-5.8, -3.1]], dtype=np.float32)
result = model.predict(test_data)
print(result)